In [1]:
#Updated Model from the initial one created 

In [2]:
#Updates include the following:
# 1. clean data
# 2. connection to a database that houses the csv file
# 3. Scaled the data
# 4. Define the model- deep neural net
# 5. Added 3 hidden layers 
# 6. Compile the model 
# 7. Train the model
# 8. Evaluate the loss and accuracy metrics


In [3]:
# Future changes:
# 1. Adding more hidden layers 
# 2. Running the model with a different number of features to see if accuracy and loss are affected
# 3. Altering aspects such as activation functions to see if model accuracy improves. 

In [4]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

In [5]:
# Upload csv
cardio_train_df = pd.read_csv('cardio_training.csv')
cardio_train_df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [6]:
# Define feature data
# features: age, gender, height, weight, ap_hi, 
# ap_lo, cholesterol, gluc, smoke, alco, active 
X = cardio_train_df.drop(['id','cardio'],1).values

In [7]:
# Define target data
# cardio (if the person has cardiovascular disease or not) 
y = cardio_train_df['cardio']

In [8]:
# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

In [10]:
# Fit the StandardScaler
X_scaler.fit(X_train)

StandardScaler()

In [11]:
# Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  15
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8

nn_model = tf.keras.models.Sequential()

In [13]:
# First hidden layer
nn_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

In [14]:
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

In [15]:
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

In [16]:
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [17]:
# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                180       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 389
Trainable params: 389
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1641/1641 [==============================] - 1s 676us/step - loss: 0.6511 - accuracy: 0.6151
Epoch 2/100
1641/1641 [==============================] - 1s 680us/step - loss: 0.5725 - accuracy: 0.7193
Epoch 3/100
1641/1641 [==============================] - 1s 670us/step - loss: 0.5596 - accuracy: 0.7280
Epoch 4/100
1641/1641 [==============================] - 1s 662us/step - loss: 0.5503 - accuracy: 0.7307
Epoch 5/100
1641/1641 [==============================] - 1s 710us/step - loss: 0.5526 - accuracy: 0.7266
Epoch 6/100
1641/1641 [==============================] - 1s 685us/step - loss: 0.5492 - accuracy: 0.7291
Epoch 7/100
1641/1641 [==============================] - 1s 703us/step - loss: 0.5426 - accuracy: 0.7349
Epoch 8/100
1641/1641 [==============================] - 1s 665us/step - loss: 0.5445 - accuracy: 0.7352
Epoch 9/100
1641/1641 [==============================] - 1s 698us/step - loss: 0.5519 - accuracy: 0.7261
Epoch 10/100
1641/1641 [==============================]

1641/1641 [==============================] - 1s 713us/step - loss: 0.5360 - accuracy: 0.7398
Epoch 79/100
1641/1641 [==============================] - 1s 667us/step - loss: 0.5407 - accuracy: 0.7356
Epoch 80/100
1641/1641 [==============================] - 1s 633us/step - loss: 0.5406 - accuracy: 0.7357
Epoch 81/100
1641/1641 [==============================] - 1s 653us/step - loss: 0.5403 - accuracy: 0.7370
Epoch 82/100
1641/1641 [==============================] - 1s 659us/step - loss: 0.5407 - accuracy: 0.7351
Epoch 83/100
1641/1641 [==============================] - 1s 656us/step - loss: 0.5391 - accuracy: 0.7383
Epoch 84/100
1641/1641 [==============================] - 1s 710us/step - loss: 0.5395 - accuracy: 0.7364
Epoch 85/100
1641/1641 [==============================] - 1s 672us/step - loss: 0.5361 - accuracy: 0.7396
Epoch 86/100
1641/1641 [==============================] - 1s 618us/step - loss: 0.5377 - accuracy: 0.7377
Epoch 87/100
1641/1641 [==============================] - 1